# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data.head(-1)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tuktoyaktuk,69.4541,-133.0374,12.00,71,75,3.60,CA,1627424896
1,Yulara,-25.2406,130.9889,14.91,55,90,0.00,AU,1627425106
2,Hilo,19.7297,-155.0900,26.85,83,90,4.12,US,1627425106
3,Sinnamary,5.3833,-52.9500,26.33,80,11,4.09,GF,1627424939
4,New Norfolk,-42.7826,147.0587,12.72,74,93,3.54,AU,1627425107
...,...,...,...,...,...,...,...,...,...
569,Ürümqi,43.8010,87.6005,25.53,28,0,3.00,CN,1627425255
570,Yatou,37.1500,122.3833,26.80,73,92,2.63,CN,1627425256
571,Chumphon,10.3333,98.8333,23.49,91,84,3.84,TH,1627425256
572,Mogok,22.9167,96.5000,18.42,98,100,0.87,MM,1627425256


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 21) & (weather_data["Max Temp"] <= 26)].dropna()
narrowed_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,Chu,43.5983,73.7614,22.70,29,0,3.73,KZ,1627425107
51,Yelizovo,53.1873,158.3805,22.08,64,0,5.00,RU,1627425119
85,Mizdah,31.4451,12.9801,21.65,67,0,6.06,LY,1627425128
99,Çerkezköy,41.2850,28.0003,24.57,64,0,5.14,TR,1627425132
101,East London,-33.0153,27.9116,22.81,25,0,5.28,ZA,1627425133
107,Vila Velha,-20.3297,-40.2925,25.54,77,0,4.63,BR,1627425135
140,Arraial do Cabo,-22.9661,-42.0278,25.98,65,0,2.06,BR,1627425143
146,Shīeli,44.1667,66.7500,23.26,26,0,4.12,KZ,1627425144
177,Chardara,41.2547,67.9692,24.35,28,0,7.62,KZ,1627425152
206,Dustlik,40.5247,68.0358,22.03,24,0,3.98,UZ,1627425160


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,Chu,KZ,43.5983,73.7614,
51,Yelizovo,RU,53.1873,158.3805,
85,Mizdah,LY,31.4451,12.9801,
99,Çerkezköy,TR,41.2850,28.0003,
101,East London,ZA,-33.0153,27.9116,
107,Vila Velha,BR,-20.3297,-40.2925,
140,Arraial do Cabo,BR,-22.9661,-42.0278,
146,Shīeli,KZ,44.1667,66.7500,
177,Chardara,KZ,41.2547,67.9692,
206,Dustlik,UZ,40.5247,68.0358,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 6: Chu.
Closest hotel in Chu is Shu Palace Hotel.
------------
Retrieving Results for Index 51: Yelizovo.
Closest hotel in Yelizovo is Yu Hotel.
------------
Retrieving Results for Index 85: Mizdah.
Closest hotel in Mizdah is فندق مزدة الكبير.
------------
Retrieving Results for Index 99: Çerkezköy.
Closest hotel in Çerkezköy is Wyndham Cerkezkoy.
------------
Retrieving Results for Index 101: East London.
Closest hotel in East London is Blue Lagoon Hotel.
------------
Retrieving Results for Index 107: Vila Velha.
Closest hotel in Vila Velha is Golden Tulip Porto Vitória.
------------
Retrieving Results for Index 140: Arraial do Cabo.
Closest hotel in Arraial do Cabo is Orlanova Boutique Hotel.
------------
Retrieving Results for Index 146: Shīeli.
Closest hotel in Shīeli is Turan Premium hotel.
------------
Retrieving Results for Index 177: Chardara.
Closest hotel in Chardara is Gostinitsa "Kyzyl Zhar".
------------
Retrieving Results for Index 206: Dustli

In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,Chu,KZ,43.5983,73.7614,Shu Palace Hotel
51,Yelizovo,RU,53.1873,158.3805,Yu Hotel
85,Mizdah,LY,31.4451,12.9801,فندق مزدة الكبير
99,Çerkezköy,TR,41.2850,28.0003,Wyndham Cerkezkoy
101,East London,ZA,-33.0153,27.9116,Blue Lagoon Hotel
107,Vila Velha,BR,-20.3297,-40.2925,Golden Tulip Porto Vitória
140,Arraial do Cabo,BR,-22.9661,-42.0278,Orlanova Boutique Hotel
146,Shīeli,KZ,44.1667,66.7500,Turan Premium hotel
177,Chardara,KZ,41.2547,67.9692,"Gostinitsa ""Kyzyl Zhar"""
206,Dustlik,UZ,40.5247,68.0358,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))